# A brief introduction to the patent system
The patent is a register, tipically a document, that to document a exclusive discovery, invention or method and aims to give to the patent holder exclusive rights over the discovery/invention.

<--TODO: Explain the T,O method of organizing patents -->

To organize the patents and find a suitable way to structure its information, a commonly used method defines a patent with 2 characteristics:
1. **Task:** the method used in the described patent. In can be compress something or agilize a effect, for example.
2. **Object:** the "target" of the task. It can be a food, a construction material or any other object that, combined with the task, defines the patent.

This method is defined by the Hallbach matrix, that defines a list of Task and Objects that can be extracted from the Title or the Resume of the patent.

# T,O Finder
The T,O Finder is the method that identifies the Task and the Object from a given patent and in this notebook we will construct a method to do such thing.

In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
from tqdm import tqdm
import json

from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA, ConversationalRetrievalChain, create_qa_with_structure_chain, create_retrieval_chain
from langchain.chains.openai_functions import create_structured_output_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts.base import BasePromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

load_dotenv()

True

In [2]:
df_triz = pd.read_csv("../../data/processed/base_efeitos_físicos_publicada_lemmatized.csv")
df_triz.head()

,TIPO DE EFEITO,TAREFA,OBJETO,EFEITO FÍSICO,SINONIMO 1 EFEITO FISICO,SINONIMO 2 EFEITO FISICO,PT Link,PT Description,Link Wiki (English),TAREFA_lemmatized
0,Aplicação,Apertar,Sólido,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array,apertar
1,Aplicação,Apertar,Sólido dividido,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array,apertar
2,Aplicação,Concentrar,Campo,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array,concentrar
3,Aplicação,Concentrar,Sólido dividido,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array,concentrar
4,Aplicação,Depositar,Sólido dividido,Matriz de Halbach,NaN,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Halbach_array,depositar


## TRIZ Vector Store Data Preparation

We are preparing text-metadata pairs for a vector store that will be used to match patents with TRIZ (Theory of Inventive Problem Solving) principles. Here's how the data is structured:

### Text Format
The text is formatted as a natural language sentence following this pattern:

`O "{TAREFA}" é um {TIPO DE EFEITO}, que no {OBJETO} causa {EFEITO FÍSICO}.`


For example:
> "O 'Aquecimento' é um Térmico, que na Água causa Aumento de temperatura."

## Metadata Structure
Each text entry has associated metadata containing four key components from TRIZ:
- **tipo_de_efeito**: The type of effect (e.g., Mechanical, Thermal)
- **tarefa**: The task or action being performed
- **objeto**: The object being affected
- **efeito_fisico**: The resulting physical effect

This structure allows for:
1. Semantic search through the text descriptions
2. Precise filtering using the metadata fields
3. Mapping between patents and existing TRIZ principles
4. Identification of new derived TRIZ relationships

The entries are created using the `build_entry()` function and then converted into Langchain Document objects for storage in the Chroma vector store.


In [ ]:
# Creating the pairs of iniputs
def build_entry(row):
    text = f'O "{row["TAREFA"]}" é um {row["TIPO DE EFEITO"]}, que no {row["OBJETO"]} causa {row["EFEITO FÍSICO"]}.'
    metadata = {
        "tipo_de_efeito": row["TIPO DE EFEITO"],
        "tarefa": row["TAREFA"],
        "objeto": row["OBJETO"],
        "efeito_fisico": row["EFEITO FÍSICO"]
    }
    return {"text": text, "metadata": metadata}

documents = [build_entry(row) for _, row in df_triz.iterrows()]
documents[:2]

[{'text': 'O "Apertar" é um Aplicação, que no Sólido causa \xa0Matriz de Halbach.',
  'metadata': {'tipo_de_efeito': 'Aplicação',
   'tarefa': 'Apertar',
   'objeto': 'Sólido',
   'efeito_fisico': '\xa0Matriz de Halbach'}},
 {'text': 'O "Apertar" é um Aplicação, que no Sólido dividido causa \xa0Matriz de Halbach.',
  'metadata': {'tipo_de_efeito': 'Aplicação',
   'tarefa': 'Apertar',
   'objeto': 'Sólido dividido',
   'efeito_fisico': '\xa0Matriz de Halbach'}}]

Now we are going to create a Vectorstore

In [ ]:
docs = [
    Document(page_content=item["text"], metadata=item["metadata"])
    for item in documents
]

embedding_model = OpenAIEmbeddings()

persist_directory = "chroma_rag_tabular"


if not os.path.exists(persist_directory):
    vectorstore = Chroma.from_documents(
        documents=docs,
        embedding=embedding_model,
        persist_directory=persist_directory,
    )
    vectorstore.persist()
else:
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding_model,
    )

persist_directory

/tmp/ipykernel_9385/3616164546.py:8: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings()
/tmp/ipykernel_9385/3616164546.py:24: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


'chroma_rag_tabular'

In [5]:
from langchain.vectorstores import Chroma

# Reabre o banco vetorial
vectorstore = Chroma(
    persist_directory="chroma_rag_tabular",
    embedding_function=embedding_model
)

# Busca por similaridade
query = "Qual tarefa pode causar aumento da temperatura?"
results = vectorstore.similarity_search(query, k=2)

for doc in results:
    print(doc.page_content)
    print("Metadados:", doc.metadata)


O "Corroer" é um Efeito, que no Sólido causa Hipertermia.
Metadados: {'efeito_fisico': 'Hipertermia', 'tarefa': 'Corroer', 'tipo_de_efeito': 'Efeito', 'objeto': 'Sólido'}
O "Aquecer" é um Efeito, que no Líquido causa Dilatação térmica.
Metadados: {'tarefa': 'Aquecer', 'tipo_de_efeito': 'Efeito', 'efeito_fisico': 'Dilatação térmica', 'objeto': 'Líquido'}


### TRIZ Schema Data Model

The TRIZ schema implements a hierarchical data model using Pydantic for structured outputs from the LLM. Here's how it's organized:

#### Base Model
- **TrizBaseModel**: Base class that implements the `derived_from` field, which tracks relationships with existing TRIZ elements

#### Component Models
All inherit from `TrizBaseModel` and add their specific fields:

- **TrizEffect**: Represents the type of effect (e.g., "Mechanical", "Thermal")
- **TrizTask**: Captures the action being performed (e.g., "Compress", "Heat")
- **TrizObject**: Describes the target object (e.g., "Water", "Metal")
- **TrizPhysicalEffect**: Records the resulting physical effect (e.g., "Temperature increase")

#### Main Schema
- **TrizSchema**: Combines all components into a complete TRIZ representation:
  - `kind_effect`: Type classification
  - `task`: Action performed
  - `object`: Target of the action
  - `physical_effect`: Resulting change

Each component can either:
- Match an existing TRIZ element (`derived_from = None`)
- Be derived from an existing element (`derived_from = "parent_element"`)

This structure enables tracking both standard TRIZ elements and new derived relationships while maintaining data validation through Pydantic.

In [ ]:
from pydantic import BaseModel

class TrizBaseModel(BaseModel):
    derived_from: str | None = None

class TrizEffect(TrizBaseModel):
    effect: str

class TrizTask(TrizBaseModel):
    task: str

class TrizObject(TrizBaseModel):
    object: str

class TrizPhysicalEffect(TrizBaseModel):
    physical_effect: str

class TrizSchema(BaseModel):
    kind_effect: TrizEffect
    task: TrizTask
    object: TrizObject
    physical_effect: TrizPhysicalEffect

In [ ]:
# 💬 Modelo LLM
# Ref.: https://python.langchain.com/docs/integrations/chat/openai/#structured-output-and-tool-calls
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

structured_llm = llm.with_structured_output(TrizSchema)

## Explanation of the Prompt Construction and QA Chain

This section of the code builds the prompt and configures the retrieval-augmented generation (RAG) chain for extracting structured TRIZ information from patent texts using a language model.

### Prompt Construction

- The `template` variable defines detailed instructions for the LLM in Portuguese. It explains:
  - The context: the model will receive a patent title and abstract.
  - The goal: connect the patent to a TRIZ relationship using the TRIZ base (object, task, effect type, and physical effect).
  - The model can create new TRIZ relationships if none fit, but must indicate the closest existing element in the `derived_from` field.
  - Rules for when to use or derive elements, and how to fill the output fields.
  - The required output format: a JSON object with four main fields (`kind_effect`, `task`, `object`, `physical_effect`), each with a value and an optional `derived_from`.
  - An example JSON output is provided, with all curly braces escaped to avoid template variable conflicts.

### PromptTemplate and Chain Setup

- The `PromptTemplate` is created with the template and the expected input variables (`context` and `question`).
- The `PydanticOutputParser` ensures the LLM output matches the `TrizSchema` structure.
- The `combine_docs_chain` is created to combine retrieved documents and the prompt for the LLM.
- The `qa_chain` is a retrieval chain that:
  - Uses the vectorstore retriever to fetch relevant TRIZ context.
  - Passes the context and the patent input to the LLM using the prompt.
  - Parses the output into the structured schema.

### Summary

This setup enables the LLM to:
- Receive a patent description.
- Retrieve relevant TRIZ principles from the vectorstore.
- Output a structured JSON mapping the patent to TRIZ elements, including any new derived elements, following strict formatting and validation rules.

In [24]:
# Build prompt
template = """A seguir, você receberá o título e o resumo de uma patente em português.
Use o contexto oriundo da base TRIZ de objeto, tarefa, tipo de efeito e efeito físico a 
seguir para conectar a patente a uma relação na TRIZ.

Você também tem a liberdade de criar novas relações baseadas na TRIZ caso identifique 
que o efeito, tarefa, objeto e efeito físico não estejam presentes na base. Nesse caso, 
o objeto de retorno deve assinalar o campo `derived_from` com o nome do elemento mais 
próximo já existente na TRIZ.

É importante que vc deve tentar criar novos elementos somente se não tiver um 
objeto, tarefa, tipo de efeito e efeito físico que se encaixem na base TRIZ. É importante
tentar priorizar um elemento que já exista na TRIZ. Só traga um novo elemento se 
realmente nenhum elemento se aproximar do contexto.

Se você não conseguir fazer uma resposta com o 
contexto, apenas diga que não sabe, não tente inventar uma resposta.

# Regras
- Se o elemento já existir na TRIZ, você deve retornar o campo
  `derived_from` como `None`.
- Se o elemento for derivado, você deve retornar `derived_from` 
  com o nome do elemento mais próximo já existente na TRIZ.
- Sempre que um componente for derivado, na sua saída o campo 
  `derived_from` deve contar o nome do elemento mais próximo 
  já existente na TRIZ.

# Formato de saída
O formato de saída deve ser um JSON com os seguintes campos:
- `effect`: `effect`, tipo de efeito (ex: "Aumento da temperatura"), além do campo
  `derived_from` que é o nome do efeito mais próximo já existente na TRIZ.
- `task`: `task` Tarefa (ex: "Aumentar a temperatura"), além do campo 
  `derived_from` que é o nome do efeito mais próximo já existente na TRIZ.
- `object`: `object` Objeto (ex: "Água"), além do campo `derived_from`
que é o nome do efeito mais próximo já existente na TRIZ.
- `physical_effect`: `physical_effect` Efeito físico (ex: "Aumento da temperatura"), 
  além do campo `derived_from` que é o nome do efeito mais próximo já existente na TRIZ.

Exemplo:
```json
{{
  "kind_effect": {{
    "effect": "string",        // Tipo do efeito (ex: "Mecânico", "Térmico")
    "derived_from": "string?"  // Opcional: efeito origem se derivado de um existente
  }},
  "task": {{
    "task": "string",         // Ação/tarefa (ex: "Comprimir", "Aquecer") 
    "derived_from": "string?" // Opcional: tarefa origem se derivada de uma existente
  }},
  "object": {{
    "object": "string",       // Objeto alvo (ex: "Água", "Metal")
    "derived_from": "string?" // Opcional: objeto origem se derivado de um existente
  }},
  "physical_effect": {{
    "physical_effect": "string", // Efeito físico (ex: "Aumento de temperatura")
    "derived_from": "string?"    // Opcional: efeito origem se derivado de um existente
  }}
}}
```

Regras:
1. Todos os campos de texto são obrigatórios
2. O campo `derived_from` deve ser:
  - `null` se o elemento já existe na base TRIZ
  - Uma string com o nome do elemento mais próximo existente se for derivado
3. Mantenha a terminologia consistente com os elementos TRIZ existentes
4. Use descrições claras e específicas para cada campo

Contexto: {context}
{input}
Resposta:"""

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

parser = PydanticOutputParser(pydantic_object=TrizSchema)
template_with_instructions = PromptTemplate(
    template=template + "\n\n{format_instructions}",
    input_variables=["context", "question"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

combine_docs_chain = create_stuff_documents_chain(llm, template_with_instructions)

qa_chain = create_retrieval_chain(retriever, combine_docs_chain)


In [25]:
df_patents = pd.read_csv("../../data/processed/patentes_inpi_english_matched.csv")
df_patents.head()

,id_pedido,data_deposito,titulo,ipc,url,resumo,classifica_ipc,titulo_english,match_top_10_title
0,BR 11 2021 018393 0,02/03/2020,TRATAMENTO DE COLISÕES EM UPLINK,H04L 1/18,https://busca.inpi.gov.br/pePI/servlet/Patente...,"A presente invenção se refere a métodos, sis...",H04L 1/18,Treatment of collisions in Uplink,"{'Move', 'Break Down', 'Change Phase', 'Separa..."
1,BR 11 2021 018071 0,02/03/2020,ALOJAMENTO DE VELA DE IGNIÇÃO COM PROTEÇÃO ANT...,H01T 13/14,https://busca.inpi.gov.br/pePI/servlet/Patente...,ALOJAMENTO DE VELA DE IGNIÇÃO COM PROTEÇÃO A...,H01T 13/14 ; H01T 13/20 ; H01T 13/32 ; H0...,"In this case, it is necessary to ensure that y...","{'Move', 'Break Down'}"
2,BR 11 2021 016947 4,02/03/2020,ANTICORPOS QUE RECONHECEM TAU,C07K 16/18,https://busca.inpi.gov.br/pePI/servlet/Patente...,ANTICORPOS QUE RECONHECEM TAU. A invenção fo...,C07K 16/18 ; G01N 33/68,Antibodies that recognize you,"{'Move', 'Break Down', 'Change Phase', 'Cool',..."
3,BR 10 2020 004169 0,02/03/2020,AQUECEDOR DE AR A LENHA COM DUPLA EXAUSTÃO PAR...,F24H 3/00,https://busca.inpi.gov.br/pePI/servlet/Patente...,AQUECEDOR DE AR A LENHA COM DUPLA EXAUSTAO P...,F24H 3/008 ; F24H 4/06,Air heater with double exhaust to be used in a...,"{'Move', 'Break Down', 'Expand', 'Separate', '..."
4,BR 11 2021 006234 3,02/03/2020,BIBLIOTECAS DE CÉLULAS ÚNICAS E NÚCLEOS ÚNICOS...,C12N 15/10,https://busca.inpi.gov.br/pePI/servlet/Patente...,BIBLIOTECAS DE CÉLULAS ÚNICAS E NÚCLEOS ÚNIC...,C12N 15/10,Unique cell libraries and unique high-end nucl...,"{'Remove', 'Break Down', 'Move', 'Concentrate'..."


In [26]:
i = 10
title = df_patents.iloc[i]["titulo"]
abstract = df_patents.iloc[i]["resumo"]

query = f"""Título: {title}
Abstract: {abstract}"""


raw_output = qa_chain.invoke({"input": query})
# raw_output is a dict with key 'result' or similar containing string output
text_output = raw_output["answer"]  # or adjust key as per your chain
# parse string output into Pydantic model
structured_output = parser.parse(text_output)

print(query)
print(text_output)
print(structured_output)
print('\n')

Título: CONJUNTO DE ESPAÇOS DE PESQUISA PARA SINAL DE ATIVAÇÃO
Abstract:   CONJUNTO DE ESPAÇOS DE PESQUISA PARA SINAL DE ATIVAÇÃO. Vários aspectos da presente revelação geralmente se referem à comunicação não cabeada. Em alguns aspectos, um equipamento do usuário (UE) pode monitorar um conjunto de espaços de pesquisa de sinal de ativação (WUS-SS) para um sinal de ativação (WUS) de canal físico compartilhado de downlink (PDCCH). O UE pode detectar o PDCCH WUS no conjunto WUS-SS baseado pelo menos em parte em monitorar o conjunto WUS-SS. Vários outros aspectos são proporcionados.
```json
{
  "kind_effect": {
    "effect": "Detectar",
    "derived_from": null
  },
  "task": {
    "task": "Monitorar",
    "derived_from": "Detectar"
  },
  "object": {
    "object": "Sinal de Ativação",
    "derived_from": null
  },
  "physical_effect": {
    "physical_effect": "Comunicação Não Cabeada",
    "derived_from": null
  }
}
```
kind_effect=TrizEffect(derived_from=None, effect='Detectar') task=Triz

### Explanation: `insert_derived_elements` Function

The `insert_derived_elements` function is responsible for identifying and storing new (derived) TRIZ elements generated by the LLM that are not present in the original TRIZ base. Here’s how it works:

1. **Purpose**  
   - To check the structured output from the LLM for any fields (`kind_effect`, `task`, `object`, `physical_effect`) that are marked as "derived" (i.e., their `derived_from` attribute is not `None`).
   - To add these new derived elements to the vectorstore for future retrieval and semantic search.

2. **How it Works**  
   - The function iterates over each of the four main TRIZ fields in the structured output.
   - For each field, it checks if the `derived_from` attribute is filled (not `None`). If so, it collects information about the derived element (field name, value, and what it was derived from).
   - If any derived elements are found, it creates a new row (dictionary) with the values for `TAREFA`, `TIPO DE EFEITO`, `OBJETO`, and `EFEITO FÍSICO` from the structured output.
   - It then uses the `build_entry` function to create a new document (with text and metadata) and adds this document to the vectorstore.
   - Finally, it prints information about each derived element added and the text that was inserted into the vectorstore.

3. **Why is this Useful?**  
   - This approach allows the system to dynamically expand the TRIZ knowledge base with new, LLM-suggested relationships, while keeping track of their origin.
   - It ensures that the vectorstore remains up-to-date and can support richer, more flexible semantic search and reasoning in future queries.

**In summary:**  
Whenever the LLM proposes a new TRIZ element (not found in the base), this function logs and stores it, making the knowledge base grow automatically as new patents are processed.

In [27]:
def insert_derived_elements(structured_output, vectorstore):
    """
    Insere elementos derivados no vectorstore e printa informações sobre os elementos adicionados.
    
    Args:
        structured_output: Output estruturado do modelo LLM
        vectorstore: Instância do Chroma para armazenar vetores
    """
    derived_elements = []
    
    # Para cada campo do schema, verifica se é derivado
    for field_name in ["kind_effect", "task", "object", "physical_effect"]:
        field = getattr(structured_output, field_name, None)
        
        # Verifica se o campo tem derived_from preenchido (não None)
        if field and field.derived_from:
            derived_elements.append({
                "field": field_name,
                "value": getattr(field, field_name.split('_')[-1]),  # get 'effect', 'task', etc
                "derived_from": field.derived_from
            })
    
    # Se encontrou elementos derivados, cria nova entrada
    if derived_elements:
        new_row = {
            "TAREFA": getattr(structured_output.task, "task", ""),
            "TIPO DE EFEITO": getattr(structured_output.kind_effect, "effect", ""),
            "OBJETO": getattr(structured_output.object, "object", ""),
            "EFEITO FÍSICO": getattr(structured_output.physical_effect, "physical_effect", "")
        }
        
        # Cria e adiciona documento
        entry = build_entry(new_row)
        doc = Document(page_content=entry["text"], metadata=entry["metadata"])
        vectorstore.add_documents([doc])
        
        # Printa informações sobre elementos derivados
        print("\nElementos derivados adicionados ao vectorstore:")
        for elem in derived_elements:
            print(f"- {elem['field']}: '{elem['value']}' (derivado de '{elem['derived_from']}')")
        print(f"Texto adicionado: {entry['text']}")

In [28]:
# Calcula 10% do tamanho total
sample_size = int(len(df_patents) * 0.1)

# Seleciona aleatoriamente 10% das linhas
df_patents_sample = df_patents.sample(n=sample_size, random_state=42)

results = []

for idx, row in tqdm(df_patents_sample.iterrows(), total=len(df_patents_sample)):
    try:
        title = row["titulo"]
        abstract = row["resumo"]
        # Chama o modelo
        query = f"""Título: {title}\nAbstract: {abstract}"""
        raw_output = qa_chain.invoke({"input": query})
        text_output = raw_output.get("answer", "")
        structured_output = parser.parse(text_output)

        insert_derived_elements(structured_output, vectorstore)

        # Salva os campos no DataFrame original
        df_patents_sample.at[idx, "kind_effect"] = getattr(structured_output.kind_effect, "effect", "")
        df_patents_sample.at[idx, "task"] = getattr(structured_output.task, "task", "")
        df_patents_sample.at[idx, "object"] = getattr(structured_output.object, "object", "")
        df_patents_sample.at[idx, "physical_effect"] = getattr(structured_output.physical_effect, "physical_effect", "")

        # Salva o resultado completo em uma lista
        results.append(structured_output.model_dump())

    except Exception as e:
        print(f"Erro no índice {idx}: {e}")
        df_patents_sample.at[idx, "kind_effect"] = "Erro"
        df_patents_sample.at[idx, "task"] = "Erro"
        df_patents_sample.at[idx, "object"] = "Erro"
        df_patents_sample.at[idx, "physical_effect"] = "Erro"
        results.append({"kind_effect": "Erro", "task": "Erro", "object": "Erro", "physical_effect": "Erro"})

# Salva os resultados
df_patents_sample.to_csv("../../data/processed/patents_inpi_llm_matched_10percent.csv", index=False)

  2%|▏         | 5/329 [00:16<17:19,  3.21s/it]

Erro no índice 2179: 'TrizPhysicalEffect' object has no attribute 'effect'


  2%|▏         | 7/329 [00:23<17:50,  3.33s/it]


Elementos derivados adicionados ao vectorstore:
- kind_effect: 'Produção de gás' (derivado de 'Produzir')
- task: 'Produzir gás liquefeito de petróleo' (derivado de 'Produzir')
- object: 'Gás liquefeito de petróleo' (derivado de 'Gás')
Texto adicionado: O "Produzir gás liquefeito de petróleo" é um Produção de gás, que no Gás liquefeito de petróleo causa Conversão de álcoois para propano e/ou butano.


  3%|▎         | 9/329 [00:28<15:00,  2.81s/it]

...

Erro no índice 1739: 'TrizPhysicalEffect' object has no attribute 'effect'


 99%|█████████▉| 326/329 [18:01<00:09,  3.33s/it]

Erro no índice 486: Invalid json output: I'm sorry, but I cannot determine a suitable TRIZ relationship for the given patent information based on the provided context.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


100%|██████████| 329/329 [18:12<00:00,  3.32s/it]


Elementos derivados adicionados ao vectorstore:
- task: 'Tratar distúrbios do sistema nervoso central' (derivado de 'Tratar distúrbios cerebrais')
- object: 'Composto de Fórmula I' (derivado de 'Composto heterocíclico')
Texto adicionado: O "Tratar distúrbios do sistema nervoso central" é um Químico, que no Composto de Fórmula I causa Interação com neurotransmissores.


In [29]:
# Análise dos campos kind_effect, task, object e physical_effect em df_patents_sample

# Função para contar repetições e mostrar os mais frequentes
def analyze_repetitions(df, column):
    counts = df[column].value_counts()
    repeated = counts[counts >= 2]
    print(f"\nCampo: {column}")
    print(f"Total de valores únicos: {counts.shape[0]}")
    print(f"Valores que se repetem pelo menos 2 vezes: {repeated.shape[0]}")
    print("Top 5 mais frequentes:")
    print(repeated.head(5))

# Campos a analisar
fields = ["kind_effect", "task", "object", "physical_effect"]

for field in fields:
    analyze_repetitions(df_patents_sample, field)


Campo: kind_effect
Total de valores únicos: 103
Valores que se repetem pelo menos 2 vezes: 33
Top 5 mais frequentes:
kind_effect
Erro                               72
Mecânico                           52
Químico                            20
Comunicação                        10
Processamento digital de imagem    10
Name: count, dtype: int64

Campo: task
Total de valores únicos: 146
Valores que se repetem pelo menos 2 vezes: 32
Top 5 mais frequentes:
task
Erro        72
Segurar     19
Detectar    16
Produzir    13
Proteger    12
Name: count, dtype: int64

Campo: object
Total de valores únicos: 174
Valores que se repetem pelo menos 2 vezes: 25
Top 5 mais frequentes:
object
Erro               72
Sólido             21
Líquido            15
Sólido dividido    14
Anticorpos          6
Name: count, dtype: int64

Campo: physical_effect
Total de valores únicos: 181
Valores que se repetem pelo menos 2 vezes: 40
Top 5 mais frequentes:
physical_effect
Erro                                       